# TP IAR 2020

In [1]:
import numpy as np
import math
import csv
import pickle
from IPython.display import clear_output

In [2]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))
 
def sigmoid_derivada(x):
    return sigmoid(x)*(1.0-sigmoid(x))
 
def tanh(x):
    return np.tanh(x)
 
def tanh_derivada(x):
    return 1.0 - x**2


In [3]:
class NeuralNetwork:
 
    #Función constructora
    def __init__(self, layers, activation='tanh'):
        #Posibles funciones de activación (Se peuden agregar otras como relu, pero las entradas van de 0 a 10, no vale la pena)
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_derivada
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_derivada
 
        # inicializo los vectores de pesos y errores de dichos pesos
        self.weights = []
        self.deltas = []
        # rando de pesos varia entre (-1,1)
        # asigno valores aleatorios a las capas
        for i in range(1, len(layers) - 1):
            r = 2*np.random.random((layers[i-1]+1, layers[i]+1)) -1
            self.weights.append(r)
        # asigno aleatorios a capa de salida
        r = 2*np.random.random( (layers[i]+1, layers[i+1])) - 1
        self.weights.append(r)
        
 

    #Función de entrenamiento
    def fit(self, X, y, learning_rate=0.0, epochs=1):
        
        # Con esto agregamos el vector unitario a la capa de entrada
        X = np.hstack((X, np.ones((X.shape[0],1))))
        
         
        for k in range(epochs):
            # Elegimos una entrada aleatoria dentro del conjunto
            i = np.random.randint(0, X.shape[0]-1)
            a = [X[i]]
            
            #Capa de entrada
            for l in range(len(self.weights)):
                # Producto escalar entre arrays de cada valor de entrada por el peso correspondiente
                dot_value = np.dot(a[l], self.weights[l])
                # Aplicar función de activación
                activation = self.activation(dot_value)
                # Agregar a "a"
                a.append(activation)
            
            # Calculo la diferencia en la capa de salida y el valor obtenido
            error = y[i] - a[-1]
            # Guardo el error en Deltas con el inverso de la función de activación
            deltas = [error * self.activation_prime(a[-1])]
            
            # Analizamos las demás capas
            for l in range(len(a) - 2, 0, -1): 
                #Calculamos el error de la misma manera
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_prime(a[l]))
            #Agregamos el delta 
            self.deltas.append(deltas)
 
            # invertimos los deltas para aplicar backpropagation
            deltas.reverse()
 
            # Comenzamos el algoritmo de backpropagation
            # 1. Multiplcamos los delta de salida con las activaciones de entrada 
            #    para obtener el gradiente del peso.
            # 2. Actualizamos el peso en base al ratio de aprendizaje multiplicado por el gradiente
            for i in range(len(self.weights)):
                #convertimos el array analizado y los deltas en 2 dimensiones. [[]]
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)
            
            # Mostrar cada cuerto porcentaje de de las iteraciones un mensaje para informar avance
            if k % (epochs/10) == 0: 
                print("Entrenando: ", int(k/epochs*100), "%", sep="")
 
    
    #Función predictiva en base a los pesos entrenados en fit()
    def predict(self, x): 
        
        # Crear vector de unos y concatenarlo a la entrada
        ones = np.atleast_2d(np.ones(x.shape[0]))
        a = np.hstack((x, np.ones((x.shape[0],1))))
        # Pasamos el vector generado por todas las capas (Teniendo en cuenta la función de activación)
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        
        #Acomodamos las salidas al formato requerido
        res = []
        for r in range(0, len(a)):
            res.append(a[r][0])
        res = np.array(res)
        # Devolvemos el generado por las capas de salida en formato binario
        # El valor fue obtenido de manera empírica, obteniendo mejores resultados que con .5 (propuesto)
        return np.heaviside(res -.49, np.zeros(res.shape))
 

    # Imprimir los pesos
    def print_weights(self):
        print("LISTADO PESOS DE CONEXIONES")
        for i in range(len(self.weights)):
            print(self.weights[i])
 
    # Obtener los errores
    def get_deltas(self):
        return self.deltas
    
    def save_params(self, nombre="params"):
        pickle_file = open(nombre + ".pickle", 'wb')
        pickle.dump(self.weights, pickle_file)
        #pickle.dump(self.weights, open(nombre + ".pickle", "wb"))
        
    def load_params(self, nombre="params"):
        pickle_file = open(nombre + ".pickle", "rb")
        self.weights = pickle.load(pickle_file)
#         with open(nombre + ".pickle", 'rb') as f:
#             file = pickle.load(f)
#             print("file 0: ", np.array(file[0]))
#             print("file 1: ", np.array(file[1]))
#             self.weights = np.append(np.atleast_2d(np.array(file[0])), np.atleast_2d(np.array(file[1])))
#             print("we: ", self.weights)

### Lectura de datos

In [4]:
# Leemos los archivos de entrenamiento y los guardamos
X = np.genfromtxt('X.csv', delimiter=',')
Y = np.genfromtxt('Y.csv', delimiter=',')

# Creamos variables para separar un 70% de los datos para entrenar y 30% para prueba interna
XRED = []
YRED = []
XTEST = []
YTEST = []
for i in range(0,20000):
    ran = np.random.rand()
    if (ran < .7):
    #if(i < 14000):
        XRED.append(X[i])
        YRED.append(Y[i])
    else:
        XTEST.append(X[i])
        YTEST.append(Y[i])

XRED =  np.array(XRED)
YRED =  np.array(YRED)
XTEST = np.array(XTEST)
YTEST = np.array(YTEST)
    

### Clasificación y evaluación

In [5]:
clasificador = NeuralNetwork([4,100,1])

In [23]:
#Datos de entrenamiento completos
clasificador.fit(X,Y,learning_rate=0.04,epochs=100000)
res = clasificador.predict(X)
# Evaluación del clasificador para los datos de entrenamiento
cant_datos = X.shape[0]
accuracy = (cant_datos - abs(res - Y).sum())/cant_datos
clear_output(wait=True)
print("Presición:", accuracy)

# También usamos este bloque para probar ajustes ligeros en la red sin tener que esperar mucho tiempo

Presición: 0.77685


In [7]:
# Creamos la instancia de la Red
clasif = NeuralNetwork([4,8,16,8,4,1])

In [11]:
#Datos separados

#Entrenamos la red con los datos reducidos (70%) 
print("Preparando la red para entrenamiento...")
clasif.fit(XRED,YRED,learning_rate=0.0,epochs=1)
clear_output(wait=True)
print("Entrenamiento finalizado...")

# Evaluamos la red neuronal para los datos de entrenamiento
res = clasif.predict(XRED)
cant_datos = XRED.shape[0]
mat_conf = [[0,0],[0,0]]
for i in range(0,XRED.shape[0]):
    mat_conf[int(YRED[i])][int(res[i])] += 1
accuracy = (cant_datos - abs(res - YRED).sum())/cant_datos
print("Precisión entrenamiento:", accuracy)
print("Matriz de confusión entrenamiento: ", mat_conf)

# Evaluamos la red neuronal para los datos de test (no cátedra)
res_test = clasif.predict(XTEST)
cant_datos_test = XTEST.shape[0]
mat_conf_test = [[0,0],[0,0]]
for i in range(0,XTEST.shape[0]):
    mat_conf_test[int(YTEST[i])][int(res_test[i])] += 1
accuracy_test = (cant_datos_test - abs(res_test - YTEST).sum())/cant_datos_test
print("Precisión test (no cátedra):", accuracy_test)
print("Matriz de confusión test (no cátedra): ", mat_conf_test)

Entrenamiento finalizado...
Precisión entrenamiento: 0.9845187986016979
Matriz de confusión entrenamiento:  [[6875, 93], [124, 6925]]
Precisión test (no cátedra): 0.9839545378572623
Matriz de confusión test (no cátedra):  [[2986, 46], [50, 2901]]


### Validación de la cátedra y exportación de resultados y parámetros

In [55]:
X_test_catedra = np.genfromtxt('X_test.csv', delimiter=',')
# preprocesamiento si es necesario...........
res_test_catedra = clasif.predict(X_test_catedra)
np.savetxt('Y_test_Grupo_08.csv', res_test_catedra, delimiter=',')
clasif.save_params("params_Grupo_08")

### Métodos Extra

### NOTA

Tener en cuenta que la red entrenada con epochs=30000000 se realizó con una memoria RAM de 16Gb... Se puede adaptar corriendo varias veces y utilizando el save/load params con iteraciones más cortas.

In [47]:
clasif.save_params("params_Grupo_08")

In [9]:
clasif.load_params("params_Grupo_08")

In [10]:
clasif.print_weights() 


LISTADO PESOS DE CONEXIONES
[[ 4.77063178e-01  2.68000079e+00 -1.40143757e+00  2.26447800e+00
   1.84022827e-02 -1.68690706e-02 -2.20446320e+00 -3.51701540e+00
   5.49004784e-03  2.46784389e+00  6.65332702e-01  7.65824173e-01
   1.45002363e+00 -1.44546430e-02  1.16016842e+00 -6.04288500e-01
  -1.37805859e+00 -2.57257389e-01  2.62846054e-01 -2.18106957e+00
  -3.30544776e-01  1.34116461e+00 -8.33668537e-01  5.23295589e-01
   7.45237993e-02  1.15561917e+00  4.59463554e-02  5.05770292e-03
   1.47201279e-02 -8.11997418e-01 -7.28917255e-01  6.30655064e-01
  -9.31184402e-01 -4.38733524e-03  6.16045442e-01 -1.47190732e+00
   1.24823268e-03  3.56215913e-02 -1.97721780e+00  1.09177769e+00
  -4.20780753e-03  1.02385533e+00 -3.30862644e+00  8.77338342e-01
   1.44571658e-01  1.38321019e+00  4.94779132e+00 -3.40612225e-01
  -3.64124149e-01  4.21180609e+00  3.27370995e-03  1.81739923e+00
   7.21239243e-04 -2.04307243e+00  5.80229430e-01 -1.13491889e-02
   1.98660478e+00 -7.14626606e-01 -4.49172276e-0

In [11]:
#Entrenamiento iterativo
rate=0.003
accuracy = 0
accuracy_test=0
ac_test_ant = 0
ac_ant = 0
vuelta = 0
while rate > 0.000001 and accuracy_test < 0.98 and accuracy - accuracy_test < .07 and vuelta < 20:
    #Entrenamos la red con los datos reducidos (70%) 
    print("Preparando la red para entrenamiento...")
    clasif.fit(XRED,YRED,learning_rate=rate,epochs=500000)
    clear_output(wait=True)
    print("Entrenamiento finalizado...")
    print("Rate: ", rate)
    print("Vuelta: ", vuelta + 1)
    # Evaluamos la red neuronal para los datos de entrenamiento
    res = clasif.predict(XRED)
    cant_datos = XRED.shape[0]
    mat_conf = [[0,0],[0,0]]
    for i in range(0,XRED.shape[0]):
        mat_conf[int(YRED[i])][int(res[i])] += 1
    accuracy = (cant_datos - abs(res - YRED).sum())/cant_datos
    print("Precisión entrenamiento:", accuracy)
    print("Matriz de confusión entrenamiento: ", mat_conf)

    # Evaluamos la red neuronal para los datos de test (no cátedra)
    res_test = clasif.predict(XTEST)
    cant_datos_test = XTEST.shape[0]
    mat_conf_test = [[0,0],[0,0]]
    for i in range(0,XTEST.shape[0]):
        mat_conf_test[int(YTEST[i])][int(res_test[i])] += 1
    accuracy_test = (cant_datos_test - abs(res_test - YTEST).sum())/cant_datos_test
    print("Precisión test (no cátedra):", accuracy_test)
    print("Matriz de confusión test (no cátedra): ", mat_conf_test)
    diff_test = accuracy_test - ac_test_ant
    diff = accuracy - ac_ant
    ac_test_ant = accuracy_test
    ac_ant = accuracy 
    
    if diff > 0.001 or diff_test > 0.001:
        rate /= 2
    else:
        rate *= 1.5
    vuelta += 1

Entrenamiento finalizado...
Rate:  1.2514114379882814e-05
Vuelta:  20
Presición entrenamiento: 0.9803613511390417
Presición test (no cátedra): 0.9726529931632483
